
**Proyecto:** Red IoT para monitoreo sísmico estructural (ESP8266 + MPU6050 + MQTT + Node-RED + MySQL)

**Versión del informe:** v0.1

**Fecha:** 28/10/2025

**Equipo:** [Completar]

**Responsables por sección:** [Completar]


# Resumen ejecutivo


1.1 **Objetivo actual**: desplegar 4–10 nodos con adquisición triaxial, sincronización NTP y publicación MQTT; backend en Ubuntu Server con Mosquitto, Node-RED y MySQL; dashboard con KPIs (PGA/PGV, espectros, alarmas).
1.2 **Estado global**:

* **Alcance**: XX% completado. @define
* **Calidad**: error preliminar ≤ 10% en magnitudes objetivo (meta); mediciones de banco en curso.
* **Confiabilidad**: reconexión MQTT y watchdog implementados en prototipo; colas locales en diseño.
* **Seguridad**: TLS en broker en pruebas; ACLs definidas.
  1.3 **Hitos cumplidos**: prototipo de firmware estable, broker y flujo base de ingesta, esquema relacional inicial.
  1.4 **Próximos hitos**: calibración en mesa vibradora o estímulo equivalente; instalación piloto en edificio; dashboard con roles.


# Alcance y entregables (trazabilidad)


## **Requerimientos clave** $\to$ **Artefactos** $\to$ **Pruebas** $\to$ **Resultado**

* Adquisición 100–200 Hz por eje $\to$ Firmware sampling + antialiasing $\to$ Prueba de aliasing y latencia $\to$ OK en banco.
* Timestamp NTP ±5 ms $\to$ Sincronización SNTP + offset $\to$ Test contra servidor NTP local $\to$ En curso.
* Publicación MQTT confiable $\to$ QoS1 + LWT + reconexión $\to$ Caída de red simulada $\to$ Pérdidas < 0.5% (objetivo).
* Persistencia $\to$ MySQL particionada por fecha $\to$ Carga 24 h sintética $\to$ En curso.
* Visualización y alertas $\to$ Node-RED Dashboard + reglas $\to$ Alarmas umbrales PGA/PGV $\to$ En curso.


# Arquitectura end-to-end (estado al corte)


## **Diagrama de bloques (visión actual)**


![alt text](Pictures/Diagrama1.png)


## **Secuencia de datos (simplificada)**

```mermaid
sequenceDiagram
  participant IMU as MPU6050
  participant MCU as ESP8266 FW
  participant MQTT as Broker
  participant NR as Node-RED
  participant DB as MySQL
  participant UI as Dashboard

  IMU->>MCU: Lectura ax, ay, az @ Fs
  MCU->>MCU: Filtro/decimado + timestamp NTP
  MCU->>MQTT: publish sensor/nX/data (QoS1)
  MQTT->>NR: mensaje entregado
  NR->>DB: INSERT batch
  NR->>UI: KPIs/Gráficas
  NR->>NR: Reglas de alerta (umbral)
```

## **Puntos de fallo y mitigaciones**

* **NTP no disponible**: buffer local con timestamp relativo; resellado al reconectar NTP.
* **Broker caído**: retención local circular + reintentos exponenciales.
* **DB saturada**: cola intermedia en Node-RED + inserts bulk + particionado por día.
* **Cortes de energía**: UPS para servidor; brown-out detection en nodos.



# Estado por componente


## Nodos (hardware)

* **Avances**: ESP-01 + GY-521 integrados; alimentación fija + Li-ion con protección; fijación mecánica preliminar.
* **Pendientes**: enclousure definitivo; soporte antivibratorio mínimo y orientación eje-edificio documentada.
* **Riesgos**: ruido eléctrico de alimentación; mitigación con LC + rutas cortas GND.

## Firmware (ESP8266/Arduino core)

* **Implementado**:

  * Muestreo a 200 Hz con decimado a 100 Hz para publicación.
  * Filtro MA/FIR ligero; watchdog; reconexión robusta MQTT; LWT.
  * Payload JSON/CBOR con esquema validado.
* **En curso**: cola local en SPIFFS cuando no hay red; NTP con corrección de offset y jitter.
* **Pruebas**: estrés 2 h continuo sin pérdidas; latencia E2E p50 180 ms, p95 350 ms.

**MQTT (propuesta de tópicos y QoS)**

* `sismo/<site>/<nodeId>/data` QoS1, no retenido.
* `sismo/<site>/<nodeId>/status` QoS1, retenido, LWT=`offline`.
* `sismo/<site>/<nodeId>/cfg` QoS1, retenido, inbound al nodo.

**Esquema de payload (JSON ejemplo)**

```json
{
  "ts_ms": 1730158805123,
  "fs_hz": 100,
  "seq": 15231,
  "ax": [0.001, -0.003, 0.002],
  "ay": [0.000, 0.001, -0.001],
  "az": [-1.001, -1.000, -0.999],
  "temp_c": 27.5,
  "vbat": 3.92
}
```

## Backend y visualización

* **Mosquitto**: instancia con TLS, usuarios, ACLs básicos; prueba de carga con `mqtt-bench` en curso.
* **Node-RED**: flujo de ingesta $\to$ validación $\to$ buffer $\to$ insert MySQL; dashboard inicial con PGA/PGV y espectro.
* **Alertas**: reglas por umbral configurable; canal de notificación pendiente (email/Telegram).

## Persistencia (MySQL)

**DDL actual (borrador operativo)**

```sql
CREATE TABLE IF NOT EXISTS imu_sample (
  id BIGINT UNSIGNED AUTO_INCREMENT PRIMARY KEY,
  site VARCHAR(32) NOT NULL,
  node_id VARCHAR(32) NOT NULL,
  ts_ms BIGINT NOT NULL,
  fs_hz SMALLINT NOT NULL,
  ax FLOAT NOT NULL, ay FLOAT NOT NULL, az FLOAT NOT NULL,
  temp_c FLOAT NULL, vbat FLOAT NULL,
  seq INT NOT NULL,
  KEY k_ts (ts_ms),
  KEY k_node_ts (node_id, ts_ms)
) PARTITION BY RANGE (ts_ms) (
  PARTITION p2025_10 VALUES LESS THAN (1767225600000)
);
```

* **Estrategia**: inserts bulk; partición diaria; retención 180 días crudos, agregados permanentes.
* **Consultas típicas**: ventana por evento, KPIs diarios, disponibilidad por nodo.

## Calibración y validación

* **Plan**: estímulo sinusoidal conocido; verificación linealidad; SNR; Allan variance si aplica.
* **Meta**: error ≤ 10% en PGA/PGV frente a referencia; reporte de incertidumbre.
* **Estado**: set-up de ensayo en armado.

## Pruebas de integración y resiliencia

* **Smoke tests**: publicación de 10 nodos sintéticos por 30 min, pérdida < 0.5%.
* **Resiliencia**: desconexión broker 5 min, recuperación con reenvío; power cycle nodo y servidor.
* **Pendiente**: fault injection de alta frecuencia de mensajes y jitter NTP.

## Instalación in situ y O&M

* **Criterios**: ubicación por piso con orientación eje X paralelo a eje largo del edificio; fijación rígida.
* **O&M**: checklist trimestral de batería, limpieza, estado mecánico; plan de contingencia por caída de red.

## Documentación y transferencia

* **Producido**: guía de despliegue preliminar; ER inicial; flujos Node-RED exportados.
* **Pendiente**: manual de mantenimiento; guía de roles/ACLs de dashboard.


# Métricas y KPIs al corte


* **Latencia E2E** (p50/p95): 180/350 ms (objetivo p95 ≤ 500 ms).

* **Disponibilidad broker**: 99.5% semanal.

* **Tasa de paquetes perdidos**: 0.4% en stress test.

* **Uso servidor**: CPU 12% promedio, RAM 1.3 GB Node-RED + MySQL.

* **Exactitud**: en banco, PGA comparado con referencia: error medio 8–12% (requiere calibración fina).



# Riesgos y mitigaciones (corto plazo)


| Riesgo                     | Impacto | Prob. | Mitigación                                                    |
| -------------------------- | ------- | ----: | ------------------------------------------------------------- |
| Deriva térmica IMU         | Medio   | Media | Calibración por temperatura; corrección en firmware           |
| Pérdidas bajo caída de red | Alto    | Media | Cola local + reintentos QoS1; persistencia temporal en SPIFFS |
| Saturación de DB           | Medio   | Media | Bulk insert, particionado, índices mínimos                    |
| Desfase NTP                | Alto    |  Baja | Servidor NTP local y monitoreo de offset                      |



# Cronograma y plan de la siguiente iteración (1 semana)


**Objetivos verificables**

* Implementar cola local en firmware con reenvío y prueba de caída 10 min.
* Integrar TLS mutual en Mosquitto y validar ACLs por tópico.
* Completar dashboard con espectro en ventana móvil y alarmas.
* Ensayo de calibración de 2 nodos y reporte de error con incertidumbre.

**Checklist de la semana**

1. Firmware: SPIFFS queue + tests de envejecimiento 2 h.
2. Broker: TLS + certificados + ACL por `sismo/<site>/<nodeId>/*`.
3. Node-RED: nodo de agregación, vista PGA/PGV, export de flujo.
4. MySQL: crear partición automática para día siguiente.
5. Calibración: protocolo, captura, cálculo SNR y sesgo.


# Anexos operativos


## Archivo de configuración Mosquitto (borrador)

**Intención:** habilitar TLS, autenticación y ACL por tópico.
**Criterio de prueba:** conectar con cliente autorizado y rechazar tópico no permitido.

```conf
listener 8883
allow_anonymous false
password_file /etc/mosquitto/passwd
cafile /etc/mosquitto/certs/ca.crt
certfile /etc/mosquitto/certs/server.crt
keyfile /etc/mosquitto/certs/server.key
require_certificate false
acl_file /etc/mosquitto/acl
persistence true
persistence_location /var/lib/mosquitto/
```

**ACL ejemplo:**

```
user nodo_01
topic write sismo/siteA/nodo_01/data
topic write sismo/siteA/nodo_01/status

user nodered_ingestor
topic read sismo/siteA/+/data
topic read sismo/siteA/+/status
topic write sismo/siteA/+/cfg
```

## Flujos Node-RED (estructura mínima)

**Intención:** validar ingestión, parseo y persistencia.
**Criterio de prueba:** llegada de 1k msgs/min sostenida 5 min sin pérdida.

```json
[
  {"type":"tab","label":"Ingesta","id":"flow_ingesta"},
  {"type":"mqtt in","z":"flow_ingesta","name":"data_in","topic":"sismo/+/+/data","qos":"1"},
  {"type":"json","z":"flow_ingesta","name":"parse"},
  {"type":"function","z":"flow_ingesta","name":"validate","func":"// validar campos y rango"},
  {"type":"batch","z":"flow_ingesta","name":"bulk","interval":"5","max":"500"},
  {"type":"mysql","z":"flow_ingesta","name":"insert_bulk","query":"INSERT ..."}
]
```

## Docker Compose (opcional)

**Intención:** orquestar Mosquitto, Node-RED, MySQL con volúmenes y backup.
**Criterio de prueba:** levantar stack en limpio, conservar datos tras reinicio.

```yaml
version: "3.8"
services:
  mosquitto:
    image: eclipse-mosquitto:2
    ports: ["8883:8883"]
    volumes:
      - ./mosquitto:/mosquitto
  nodered:
    image: nodered/node-red:latest
    ports: ["1880:1880"]
    volumes:
      - ./nodered:/data
  mysql:
    image: mysql:8
    environment:
      - MYSQL_ROOT_PASSWORD=strongpass
      - MYSQL_DATABASE=sismo
    ports: ["3306:3306"]
    volumes:
      - ./mysql:/var/lib/mysql
```


# Requerimientos abiertos y supuestos


* **Falta definir**: cantidad final de nodos por planta y ubicación exacta; política de retención definitiva.
* **Supuestos**: Fs efectivo de publicación 100 Hz; latencia objetivo p95 ≤ 500 ms; retención crudos 180 días.



# Próximas entregas y criterios de aceptación


* **Piloto 4 nodos in situ:** 1) disponibilidad broker ≥ 99% semana, 2) pérdidas < 1%, 3) error PGA/PGV ≤ 10% vs referencia.
* **Informe técnico de investigación:** metodología de calibración, resultados, incertidumbre, lecciones aprendidas.
* **Guía de transferencia/mantenimiento:** instalación, O&M, roles, seguridad.



# Matriz de cambios (desde último informe)


| ID cambio | Descripción       | Motivo                 | Impacto                | Estado    |
| --------- | ----------------- | ---------------------- | ---------------------- | --------- |
| CH-07     | Decimado a 100 Hz | Reducir ancho de banda | Menor carga y latencia | Aplicado  |
| CH-08     | CBOR opcional     | Payload más compacto   | Menor uso de red       | En prueba |


# Apéndice: Plan mínimo de pruebas automáticas (firmware)


**Intención:** asegurar estabilidad de muestreo y reconexión.
**Criterio de prueba:** pasar 100% de casos en banco.

* Temporización Fs ±0.5% en 15 min.
* Reconexión MQTT < 3 s tras caída simulada.
* Integridad de secuencia `seq` sin saltos tras reconexión.


# Solicitudes al comité/directores (bloqueadores)


1. Autorización de instalación piloto y acceso a Ethernet/PoE donde aplique.
2. Presupuesto para UPS y caja estanca IP54 por nodo.
3. Acceso a mesa vibradora o protocolo alternativo de estímulo.





### Notas de uso

* Este informe es plantilla operativa: reemplazar campos entre corchetes y completar percentiles/métricas con datos reales.
* Mantener numeración y trazabilidad para comparación entre cortes.

**Fin del informe.**
